In [1]:
# Import all required libraries
import pandas
from pandas import *
import numpy
from datetime import datetime

# Initialize values

env = None
sample_size = None
# Set env, if env = test, will only be run locally and display the result
env = "prod"
env = "test"

nb_models = 10

nb_year_min = 1
nb_year_max = 0

nb_week_min = 15
nb_week_max = 0
# Number of value on which to train, if null, train on all value
sample_size = 480000

In [2]:
# Read training data + test data
df_data = pandas.read_csv("../input/train.csv")
df_test = pandas.read_csv("../input/test.csv")


# Display basic information
display(df_data.head(5))
print(df_data.describe())
df_data.columns

df_test.describe()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


               store           item          sales
count  913000.000000  913000.000000  913000.000000
mean        5.500000      25.500000      52.250287
std         2.872283      14.430878      28.801144
min         1.000000       1.000000       0.000000
25%         3.000000      13.000000      30.000000
50%         5.500000      25.500000      47.000000
75%         8.000000      38.000000      70.000000
max        10.000000      50.000000     231.000000


,id,store,item
count,45000.000000,45000.000000,45000.00000
mean,22499.500000,5.500000,25.50000
std,12990.525394,2.872313,14.43103
min,0.000000,1.000000,1.00000
25%,11249.750000,3.000000,13.00000
50%,22499.500000,5.500000,25.50000
75%,33749.250000,8.000000,38.00000
max,44999.000000,10.000000,50.00000


In [3]:
if env == "test":
    # Take the last 3 months of 2017 as testing data
    df_test = df_data[df_data.date >= '2017-10-01']
    # Remove the last 3 months, as it would not be fair to train on those
    df_train = df_data[df_data.date < '2017-10-01']
    if nb_week_max > 0 or nb_year_max >0:
        df_train = df_tain[df_train.date >= '2015-01-01']
else:
    if nb_week_max > 0 or nb_year_max >0:
        df_train = df_data[df_data.date >= '2015-01-01']
    else:
        df_train = df_data
    df_test['sales'] = 0

# Only select a small sample, faster local testing
if sample_size is not None and sample_size > 0:
    df_train = df_train.sample(sample_size)
    
display(df_train.head(5))
df_test.describe()

,date,store,item,sales
38405,2013-03-01,2,3,42
406022,2014-10-13,3,23,24
574476,2016-01-18,5,32,20
737321,2016-12-14,4,41,22
308369,2017-05-21,9,17,56


,store,item,sales
count,46000.000000,46000.000000,46000.000000
mean,5.500000,25.500000,54.677543
std,2.872313,14.431027,28.485947
min,1.000000,1.000000,3.000000
25%,3.000000,13.000000,32.000000
50%,5.500000,25.500000,50.000000
75%,8.000000,38.000000,73.000000
max,10.000000,50.000000,174.000000


In [4]:
# Add time to df_data
from datetime import timedelta
from dateutil.relativedelta import relativedelta

# This can be quite slow, but is important for next steps

if nb_week_max > 0 or nb_year_max >0:
    df_data = df_data.assign(time = df_data.date.apply(lambda x: datetime.strptime(x, "%Y-%m-%d")))

In [5]:
# Let's add the value for week 15 to 20
for i in range(nb_week_min, nb_week_min+nb_week_max):
    print("Adding weeks")
    df_data['weekplus{0}'.format(i)] = (df_data['time'] + timedelta(days=i*7)).astype(str)

for i in range(nb_year_min, nb_year_min+nb_year_max):
    print("Adding years")
    df_data['yearplus{0}'.format(i)] = (df_data['time'] + pandas.DateOffset(years=i)).astype(str)

df_data.head(5)
df_test.describe()

,store,item,sales
count,46000.000000,46000.000000,46000.000000
mean,5.500000,25.500000,54.677543
std,2.872313,14.431027,28.485947
min,1.000000,1.000000,3.000000
25%,3.000000,13.000000,32.000000
50%,5.500000,25.500000,50.000000
75%,8.000000,38.000000,73.000000
max,10.000000,50.000000,174.000000


In [6]:
# First preparation step, adding some values to both dataframe
def prepare_initial(df, df_data, nb_year_max = 2, nb_week_max = 5):
    for i in range(nb_year_min, nb_year_min+nb_year_max):
        df = df.merge(df_data[['yearplus{0}'.format(i), 'store', 'item', 'sales']], how='inner', 
                           left_on=['date', 'store', 'item'],
                           right_on=['yearplus{0}'.format(i), 'store', 'item'],
                           suffixes=('', '_saleminus{0}year'.format(i)))
        
        df['sales_saleminus{0}year'.format(i)] = df['sales_saleminus{0}year'.format(i)].fillna(0)

        df = df.drop(columns=['yearplus{0}'.format(i)])


    for i in range(nb_week_min, nb_week_min+nb_week_max):
        df = df.merge(df_data[['weekplus{0}'.format(i), 'store', 'item', 'sales']], how='inner', 
                           left_on=['date', 'store', 'item'],
                           right_on=['weekplus{0}'.format(i), 'store', 'item'],
                           suffixes=('', '_saleminus{0}week'.format(i)))
        
        df['sales_saleminus{0}week'.format(i)] = df['sales_saleminus{0}week'.format(i)].fillna(0)
        
        df = df.drop(columns=['weekplus{0}'.format(i)])

    print("Start")
    return df
   

df_train = prepare_initial(df_train, df_data, nb_year_max, nb_week_max)
df_test = prepare_initial(df_test, df_data, nb_year_max, nb_week_max)

if env != "test":
    # Sometimes, we have duplicates, so they should be removed:
    df_test = df_test.drop_duplicates(subset='id', keep="last")
df_train.sample(5)
df_test.describe()

Start
Start


,store,item,sales
count,46000.000000,46000.000000,46000.000000
mean,5.500000,25.500000,54.677543
std,2.872313,14.431027,28.485947
min,1.000000,1.000000,3.000000
25%,3.000000,13.000000,32.000000
50%,5.500000,25.500000,50.000000
75%,8.000000,38.000000,73.000000
max,10.000000,50.000000,174.000000


In [7]:
# Second preparation step
# We need to add all might be useful information from df_test and df_train
# Extracting some variable from date

def prepare_data(df, df_data):
    # Add time column, easier for later step
    df = df.assign(time = df.date.apply(lambda x: datetime.strptime(x, "%Y-%m-%d")))
    
    # nb days since the beginning of the data, as traffic grows by store
    df = df.assign(days = df.time.apply(lambda x: (x - datetime(2012,12,31)).days))
    
    # Week day should be used 
    df = df.assign(weekday = df.time.apply(lambda x: x.weekday()))
    
    df = df.assign(dom = df.time.apply(lambda x: x.day))
    
    df = df.assign(cw = df.time.apply(lambda x: x.isocalendar()[1]))

    df = df.assign(month = df.time.apply(lambda x: x.month))
    
    df = pandas.get_dummies(df, prefix=['store', 'item', 'dom', 'cw', 'weekday', 'month'], 
                            columns=['store', 'item', 'dom', 'cw', 'weekday', 'month'])
    
    # Should we give more information? Like nb sales previous X weeks?
    # Previous sales of last 2/3 year at the same date (bank holiday, black friday, etc.)
    # Probably yes as this information is available, would be a good try
    return df

df_train = prepare_data(df_train, df_data)
df_test = prepare_data(df_test, df_data)

# Initialize column that do not exist in test with value 0, to avoid dummy not creating enough columns
# For instance, non existing month like month_5 will not exist in test set
for train_col in df_train.columns.values:
    if train_col not in df_test.columns.values:
        df_test[train_col] = 0

# Also train the other way around
for test_col in df_test.columns.values:
    if test_col not in df_train.columns.values:
        df_train[test_col] = 0
        
display(df_train.head(5))
display(df_test.head(5))

,date,sales,time,days,store_1,store_2,store_3,store_4,store_5,store_6,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
38405,2013-03-01,42,2013-03-01,60,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
406022,2014-10-13,24,2014-10-13,651,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
574476,2016-01-18,20,2016-01-18,1113,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
737321,2016-12-14,22,2016-12-14,1444,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
308369,2017-05-21,56,2017-05-21,1602,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


,date,sales,time,days,store_1,store_2,store_3,store_4,store_5,store_6,...,cw_53,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9
1734,2017-10-01,21,2017-10-01,1735,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1735,2017-10-02,12,2017-10-02,1736,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1736,2017-10-03,18,2017-10-03,1737,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1737,2017-10-04,15,2017-10-04,1738,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1738,2017-10-05,20,2017-10-05,1739,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Generate our training/validation datasets
from sklearn import model_selection

# Name of the result column
result_cols = ['sales']
result_excl_cols = 'sales_'
# Removing input_cols = ['store', 'item',
# dom, cw, 

# best model contained:
# days, store, item, weekday, month, cw, dom
input_cols = [
#    'sales_',
    'store_',
    'item_',
    #'day', always out
    #'day_',
    #'weekday', always out
    'dom_',
    'cw_',
    'weekday_',
    #'month', always out
    'month_',
    
    'days'
]

# Get the final values
def get_values(df, cols=[], excl_cols = "doqwidjoqwidjqwoidjqwoidjqwodijqw"):
    columns = df.columns.values
    # Remove all columns that are not inside the list
    for column in columns:
        find = False
        if column.startswith(excl_cols):
            print("Ignoring {0}".format(column))
        else:
            for col in cols:
                if column.startswith(col):
                    find = True
        if not find:
            df = df.drop(columns=[column])
    new_order = sorted(df.columns.values)
    print(new_order)
    # Same order for both training and testing set
    df = df[new_order]
    return df.values

X_train = get_values(df_train, input_cols)
Y_train = get_values(df_train, result_cols, result_excl_cols).ravel()
X_test = get_values(df_test, input_cols)

# In test env, we calculate it for the test only
if env == "test":
    Y_test = get_values(df_test, result_cols, result_excl_cols).ravel()

['cw_1', 'cw_10', 'cw_11', 'cw_12', 'cw_13', 'cw_14', 'cw_15', 'cw_16', 'cw_17', 'cw_18', 'cw_19', 'cw_2', 'cw_20', 'cw_21', 'cw_22', 'cw_23', 'cw_24', 'cw_25', 'cw_26', 'cw_27', 'cw_28', 'cw_29', 'cw_3', 'cw_30', 'cw_31', 'cw_32', 'cw_33', 'cw_34', 'cw_35', 'cw_36', 'cw_37', 'cw_38', 'cw_39', 'cw_4', 'cw_40', 'cw_41', 'cw_42', 'cw_43', 'cw_44', 'cw_45', 'cw_46', 'cw_47', 'cw_48', 'cw_49', 'cw_5', 'cw_50', 'cw_51', 'cw_52', 'cw_53', 'cw_6', 'cw_7', 'cw_8', 'cw_9', 'days', 'dom_1', 'dom_10', 'dom_11', 'dom_12', 'dom_13', 'dom_14', 'dom_15', 'dom_16', 'dom_17', 'dom_18', 'dom_19', 'dom_2', 'dom_20', 'dom_21', 'dom_22', 'dom_23', 'dom_24', 'dom_25', 'dom_26', 'dom_27', 'dom_28', 'dom_29', 'dom_3', 'dom_30', 'dom_31', 'dom_4', 'dom_5', 'dom_6', 'dom_7', 'dom_8', 'dom_9', 'item_1', 'item_10', 'item_11', 'item_12', 'item_13', 'item_14', 'item_15', 'item_16', 'item_17', 'item_18', 'item_19', 'item_2', 'item_20', 'item_21', 'item_22', 'item_23', 'item_24', 'item_25', 'item_26', 'item_27', 'ite

In [9]:
df_test.describe()

,sales,days,store_1,store_2,store_3,store_4,store_5,store_6,store_7,store_8,...,cw_53,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9
count,46000.000000,46000.000000,46000.000000,46000.000000,46000.000000,46000.000000,46000.000000,46000.000000,46000.000000,46000.000000,...,46000.0,46000.0,46000.0,46000.0,46000.0,46000.0,46000.0,46000.0,46000.0,46000.0
mean,54.677543,1780.500000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,28.485947,26.556832,0.300003,0.300003,0.300003,0.300003,0.300003,0.300003,0.300003,0.300003,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,3.000000,1735.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,32.000000,1757.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,50.000000,1780.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,73.000000,1803.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,174.000000,1826.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Normalize the data


X_all = [x + y for x, y in zip(X_train, X_test)]
#print(len(X_all))
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler() 

# Don't cheat - fit only on training data
# Def adding x_train + X_test + X_validation to fit all of them
scaler.fit(X_train)  

X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test) 

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [11]:
# Custom function to calculate the SMAPE
def get_smape(Y_validation, Y_validation_predict):
    result = 0
    for i in range(0, len(Y_validation)):
        result += (abs(Y_validation[i] - Y_validation_predict[i]))/(abs(Y_validation[i])+abs(Y_validation_predict[i]))
    return result / len(Y_validation) * 200

In [12]:
# Import algorithm
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import *
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

models = []

for i in range(5, 5 + nb_models):
    models.append(('MLPRegressor_adam_{0}'.format(i), MLPRegressor(hidden_layer_sizes=(8,),  activation='relu', solver='adam', alpha=0.001, batch_size='auto',
    learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=1000, shuffle=True,
    random_state=i, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True,
    early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)))
    
    
# High value until first model get solved
max_score = 10000
best_model = "UNKNOWN"

res = []
# Testing all models, one by one
for name, model in models:
    print("Executing for model {0}".format(name))
    time_start = datetime.now()

    # Training the model
    model.fit(X_train, Y_train)
    
    print("Finish fit for {0}".format(name))

    Y_test_result = model.predict(X_test)
    res.append(Y_test_result)
    if env == "test":
        # We can calculate the avg error
        score = get_smape(Y_test, Y_test_result)
        print("Model {0} got score of {1}, time: {2}".format(name, score, datetime.now() - time_start))
    #else:
        # Let's write an output file, with the name of the model
        #print("Writing output file {0}.csv for model {0}".format(name))
        
        #df_test['sales'] = Y_test_result
        #result_df = df_test[['id', 'sales']]
        #result_df['sales'] = Y_test_result
        
        #result_df.to_csv("{0}.csv".format(name), index=False)

Executing for model MLPRegressor_adam_5
Finish fit for MLPRegressor_adam_5
Model MLPRegressor_adam_5 got score of 12.321950295145886, time: 0:00:51.247517
Executing for model MLPRegressor_adam_6
Finish fit for MLPRegressor_adam_6
Model MLPRegressor_adam_6 got score of 12.430211739267738, time: 0:02:31.057160
Executing for model MLPRegressor_adam_7
Finish fit for MLPRegressor_adam_7
Model MLPRegressor_adam_7 got score of 12.635766165890841, time: 0:02:44.901704
Executing for model MLPRegressor_adam_8
Finish fit for MLPRegressor_adam_8
Model MLPRegressor_adam_8 got score of 12.369089878541184, time: 0:01:03.371093
Executing for model MLPRegressor_adam_9
Finish fit for MLPRegressor_adam_9
Model MLPRegressor_adam_9 got score of 12.580483912062554, time: 0:02:17.353977
Executing for model MLPRegressor_adam_10
Finish fit for MLPRegressor_adam_10
Model MLPRegressor_adam_10 got score of 12.356745983979396, time: 0:02:44.298713
Executing for model MLPRegressor_adam_11
Finish fit for MLPRegresso

In [13]:
# For all result in res, if test, display the result, if not, write it to a file
final_res = []
nb_variable = len(res[0])
for variable in range(0, nb_variable):
    final_res.append(0.0)
    for i in range(0, len(res)):
        final_res[variable] += res[i][variable]
    final_res[variable] = final_res[variable] / len(res)

if env == "test":
    # We can calculate the avg error
    score = get_smape(Y_test, final_res)
    print("avg model got score of {0}".format(score))
else:
    print("Writing output file merged.csv".format(name))

    df_test['sales'] = final_res
    result_df = df_test[['id', 'sales']]
    result_df['sales'] = final_res

    result_df.to_csv("merged.csv".format(name), index=False)

avg model got score of 12.249021480529244
